In [1]:
%load_ext lab_black

Step: Load data

In [20]:
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
import numpy as np
import pandas as pd
import gower
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
)


path_data = here("./data")
os.chdir(path_data)

data_draft_kings_original = pd.read_csv("draft_kings_2021_2022.csv")
data_draft_kings = pd.read_parquet("data_draft_kings_analysis.parquet")
data_draft_kings_analysis = data_draft_kings.drop(columns=["week"])

In [22]:
data_draft_kings_original.query("Name == 'Donovan Jones'")

,Position,Name,Points,Salary,Reduction,Value,Ownership %,Place,week
167,WR,Donovan Jones,29.1,3500,3.5,8.31,0.8%,1,6


In [12]:
data_draft_kings_mean = (
    data_draft_kings_analysis.median()
    .reset_index()
    .rename(columns={"index": "attribute", 0: "median"})
    .round(2)
)

In [24]:
data_draft_kings_mean.to_csv("donovan_jones.csv")

In [3]:
# Compute Gower distance matrix
distance_matrix = gower.gower_matrix(data_draft_kings_analysis)

In [4]:
data_clusters = [2, 3, 4, 5, 6]

In [5]:
labels_out = []

for x in range(len(data_clusters)):
    # Create the hierarchical agglomerative clustering model
    clustering_in = AgglomerativeClustering(
        n_clusters=data_clusters[x], affinity="precomputed", linkage="complete"
    )

    clustering_in.fit(distance_matrix)
    labels = clustering_in.labels_

    labels_out.append(labels)

In [6]:
accuracy_out = []
for x in range(len(labels_out)):
    accuracy_in = silhouette_score(data_draft_kings_analysis, labels_out[x])
    accuracy_out.append(accuracy_in)

In [7]:
accuracy_out

[0.288035499192513,
 0.04312217895576275,
 -0.030924717001696758,
 -0.08095817523341031,
 -0.18009020473223336]

Step: Save 

In [8]:
labels = pd.DataFrame(labels_out[1]).rename(columns={0: "labels"})

In [9]:
data_draft_kings_analysis_labels = pd.concat(
    [data_draft_kings_analysis, labels], axis=1
)

In [10]:
data_draft_kings_analysis_labels.columns

Index(['points', 'salary', 'reduction', 'value', 'ownership_%', 'WR', 'RB',
       'QB', 'TE', 'FLEX',
       ...
       'buccaneers', 'henry_ruggs', 'ricky_seals_jones', 'noah_fant',
       'donovan_jones', 'steelers', '1', '2', '3', 'labels'],
      dtype='object', length=220)

In [11]:
data_draft_kings_cluster = (
    data_draft_kings_analysis_labels.groupby("labels").mean().reset_index().round(2)
)

data_draft_kings_cluster.to_csv("data_draft_kings_cluster.csv")